In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import function
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import *
from keras.initializers import glorot_uniform
from matplotlib.pyplot import imshow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import h5py
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
def bn_relu(X, axis, activation):
  X_out = BatchNormalization(axis=axis)(X)
  X_out = Activation(activation)(X_out)

  return X_out

In [ ]:
def res_block(X, f, filters):
  F1, F2, F3 = filters
  X_skipped = X

  #Layer 1
  X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1,1), padding="valid", kernel_initializer = glorot_uniform(seed=0))(X)
  X = bn_relu(X, 3, "relu")

  #Layer 2
  X = Conv2D(filters=F2, kernel_size=(f,f), strides=(1,1), padding="same", kernel_initializer=glorot_uniform(seed=0))(X)
  X = bn_relu(X, 3, "relu")

  #Layer 3
  X = Conv2D(filters=F3, kernel_size=(1,1), strides=(1,1), padding="valid", kernel_initializer=glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=3)(X)

  X = Add()([X, X_skipped])
  X = Activation("relu")(X)

  return X

In [ ]:
def conv_block(X, f, s, filters):
  F1, F2, F3 = filters
  x_short_orig = X

  #Layer 1
  X = Conv2D(filters=F1, kernel_size=(1,1), strides=(s,s), padding="valid", kernel_initializer = glorot_uniform(seed=0))(X)
  X = bn_relu(X, 3, "relu")

  #Layer 2
  X = Conv2D(filters=F2, kernel_size=(f,f), strides=(1,1), padding="same", kernel_initializer = glorot_uniform(seed=0))(X)
  X = bn_relu(X, 3, "relu")

  #Layer 3
  X = Conv2D(filters=F3, kernel_size=(1,1), strides=(1,1), padding="valid", kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=3)(X)

  #Shortcut
  X_short = Conv2D(filters=F3, kernel_size=(1,1), strides=(s,s), padding="valid", kernel_initializer = glorot_uniform(seed=0))(x_short_orig)
  X_short = BatchNormalization(axis=3)(X_short)

  X = Add()([X_short, X])
  X = Activation("relu")(X)

  return X

In [ ]:
def ResNet50(input_shape, output_classes):

  X_input = Input(shape=input_shape)

  X = ZeroPadding2D((3,3))(X_input)

  #Stage 1
  X = Conv2D(filters=64, kernel_size=(7,7), strides=(2,2), kernel_initializer=glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=3)(X)
  X = MaxPool2D(pool_size=(3,3), strides=(2,2))(X)

  #Stage 2
  X = conv_block(X, 3, 1,[64, 64, 256])
  X = res_block(X, 3, [64,64,256])
  X = res_block(X, 3, [64,64,256])

  #Stage 3
  X = conv_block(X, 3, 2, [128,128,512])
  X = res_block(X, 3, [128,128,512])
  X = res_block(X, 3, [128,128,512])
  X = res_block(X, 3, [128,128,512])

  #Stage 4
  X = conv_block(X, 3, 2, [256, 256, 1024])
  X = res_block(X, 3, [256,256,1024])
  X = res_block(X, 3, [256,256,1024])
  X = res_block(X, 3, [256,256,1024])
  X = res_block(X, 3, [256,256,1024])
  X = res_block(X, 3, [256,256,1024])

  #Stage 5
  X = conv_block(X, 3, 2, [512,512,2048])
  X = res_block(X, 3, [256, 256, 2048])
  X = res_block(X, 3, [256, 256, 2048])

  X = AveragePooling2D(pool_size=(2,2), padding="same")(X)
  X = Flatten()(X)
  X = Dense(output_classes, activation="softmax", kernel_initializer=glorot_uniform(seed=0))(X)

  model = Model(inputs=X_input, outputs=X, name="ResNet50")

  return model

In [ ]:
model = ResNet50(input_shape=(64,64,3), output_classes=6)
model.compile(optimizer="adam", loss="CategoricalCrossentropy", metrics=["accuracy"])

In [ ]:
train_dataset = h5py.File('/content/drive/MyDrive/ML Datasets/finger-signs/Signs_Data_Training.h5', "r")
train_set_x = np.array(train_dataset["train_set_x"][:]) # your train set features
train_set_y = np.array(train_dataset["train_set_y"][:]) # your train set labels

test_dataset = h5py.File('/content/drive/MyDrive/ML Datasets/finger-signs/Signs_Data_Testing.h5', "r")
test_set_x = np.array(test_dataset["test_set_x"][:]) # your test set features
test_set_y = np.array(test_dataset["test_set_y"][:]) # your test set labels

classes = np.array(test_dataset["list_classes"][:]) # the list of classes

#Converting to 2D Numpy arrays
train_set_y = train_set_y.reshape((1, train_set_y.shape[0]))
test_set_y = test_set_y.reshape((1, test_set_y.shape[0]))

#Converting to One hot arrays
Y_test = np.zeros((test_set_y.size, test_set_y.max()+1))
Y_test[np.arange(test_set_y.size), test_set_y] = 1

Y_train = np.zeros((train_set_y.size, train_set_y.max()+1))
Y_train[np.arange(train_set_y.size), train_set_y] = 1

X_train = train_set_x / 255
X_test = test_set_x / 255

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

print("number of training examples = " + str(X_train.shape[0]))
print("number of test examples = " + str(X_test.shape[0]))
print("number of validation examples = " + str(X_val.shape[0]))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape: " + str(X_test.shape))
print("Y_test shape: " + str(Y_test.shape))
print("X_val shape: " + str(X_val.shape))
print("Y_val shape: " + str(Y_val.shape))

In [ ]:
model.fit(X_train, Y_train, batch_size=32, epochs=4, validation_data=(X_val, Y_val))

In [ ]:
test = model.evaluate(X_test, Y_test)
print("Loss = ", str(test[0]))
print("Accuracy = ", str(test[1]))